In [11]:
src = './tests/blas/level1/test_sgemm.c'
# src = './src/blas/level3/sgemm.c'
with open(src, 'r') as srcfile:
    srccode = srcfile.read()

In [16]:
kind = 'd'
fn = 'sgemm'

In [17]:
types = {'h': 16, 's': 32, 'd': 64, 'q': 128}
types_ = {'d': 'double', 'h':'uint16_t'}
values = {
    'h': {
            '-5.0f' : '0xc500 ',
            '-4.0f' : '0xc400 ',
            '-3.0f' : '0xc200 ',
            '-2.0f' : '0xc000 ',
            '-1.0f' : '0xbc00 ',
             '0.0f' : '0x0 ',
             '1.0f' : '0x3c00 ',
             '2.0f' : '0x4000 ',
             '3.0f' : '0x4200 ',
             '4.0f' : '0x4400 ',
             '5.0f' : '0x4500 ',
            '10.0f' : '0x4900 ',
            '12.0f' : '0x4a00 ',
            '16.0f' : '0x4c00 ',
            '20.0f' : '0x4d00 ',
            '24.0f' : '0x4e00 ',
            '30.0f' : '0x4f80 ',
            '32.0f' : '0x5000 ',
            '36.0f' : '0x5080 ',
            '40.0f' : '0x5100 ',
            '48.0f' : '0x5200 ',
            '50.0f' : '0x5240 ',
            '60.0f' : '0x5380',
            '64.0f' : '0x5400'
    },
    'd': {
            '-5.0f' : '-5.0',
            '-4.0f' : '-4.0',
            '-3.0f' : '-3.0',
            '-2.0f' : '-2.0',
            '-1.0f' : '-1.0',
             '1.0f' : '1.0',
             '2.0f' : '2.0',
             '3.0f' : '3.0',
             '4.0f' : '4.0',
            '10.0f' : '10.0',
            '12.0f' : '12.0',
            '16.0f' : '16.0',
            '20.0f' : '20.0',
            '24.0f' : '24.0',
            '30.0f' : '30.0',
            '32.0f' : '32.0',
            '36.0f' : '36.0',
            '40.0f' : '40.0',
            '48.0f' : '48.0',
            '50.0f' : '50.0',
            '60.0f' : '60.0',
            '64.0f' : '64.0',
             '0.0f' : '0.0',
             '5.0f' : '5.0',
    }
}
asserts = {
    'h': 'assert_ushort',
    'd': 'assert_ullong',
    'q': 'assert_ullong'
}

In [18]:
trgcode = srccode.replace('32', repr(types[kind]))
trgcode = trgcode.replace(fn, kind+fn[1:])
trgcode = trgcode.replace('svec', kind+'vec')
trgcode = trgcode.replace('stemp', kind+'temp')
trgcode = trgcode.replace('S', kind.upper())
trgcode = trgcode.replace(kind.upper()+'B', 'SB')
trgcode = trgcode.replace('SX', kind.upper()+'X')
trgcode = trgcode.replace('SX', kind.upper()+'Y')
trgcode = trgcode.replace('(float)', '('+types_[kind]+')')
trgcode = trgcode.replace('float[]', types_[kind]+'[]')
trgcode = trgcode.replace('nan_unify_s', 'nan_unify_'+kind)
trgcode = trgcode.replace('assert_ulong', asserts[kind])
for key in values[kind]:
    trgcode = trgcode.replace(key, values[kind][key])

In [19]:
print(trgcode)

#include "test.h"
#include <stdio.h>

MunitResult test_dgemm_0_row(const MunitParameter params[],
                             void *user_data) {
    const char transA = 'N';   // No transpose
    const char transB = 'N';   // No transpose
    const float64_t alpha = { SB_REAL64_ONE };  // Dcalar alpha
    const float64_t beta = { SB_REAL64_ZERO };  // Dcalar beta

    const uint64_t M = 2;     // Number of rows in A
    const uint64_t N = 2;     // Number of columns in A
    const uint64_t P = 2;     // Number of columns in B

    // 2x2 matrix A (row-major order)
    //   [1.0 2.0]
    //   [3.0 4.0]
    float64_t* A = dvec((double[]){1.0, 2.0, 3.0, 4.0}, M*N);

    // 2x2 matrix B (row-major order)
    //   [5.0 6.0]
    //   [7.0 8.0]
    float64_t* B = dvec((double[]){5.0, 6.0f, 7.0f, 8.0f}, N*P);

    // 2x2 matrix C (row-major order)
    //   [0.0 0.0]
    //   [0.0 0.0]
    float64_t* C = dvec((double[]){0.0, 0.0, 0.0, 0.0}, M*P);

    // Leading dimensions
    const uint64_t l